In [47]:
import random
import string
import pprint

# dynamic hash table with linear probing 
class DynamicHashTable:
    def __init__(self, initial_size=8):  # Default table size
        self.size = initial_size  
        self.table = [None] * self.size  # Array to hold key-value pairs
        self.num_elements = 0  # Number of elements inserted

    def _load_factor(self):
        return self.num_elements / self.size

    def _resize(self, new_size):
        old_table = self.table
        self.size = new_size
        self.table = [None] * self.size
        self.num_elements = 0

        for entry in old_table:
            if entry is not None:
                self.insert(entry[0], entry[1])

    def insert(self, key, value):
        index = myhash(key, self.size)

        while self.table[index] is not None:  # Linear probing
            if self.table[index][0] == key:  # Update existing key
                self.table[index] = (key, value)
                return
            index = (index + 1) % self.size

        self.table[index] = (key, value)
        self.num_elements += 1

        # Check load factor and resize if necessary
        if self._load_factor() >= 4 / 5:
            self._resize(self.size * 2)

    def delete(self, key):
        index = myhash(key, self.size)

        while self.table[index] is not None:
            if self.table[index][0] == key:  # key found
                self.table[index] = None
                self.num_elements -= 1

                # Rehash the cluster of entries after the removed item
                next_index = (index + 1) % self.size
                while self.table[next_index] is not None:
                    entry = self.table[next_index]
                    self.table[next_index] = None
                    self.num_elements -= 1
                    self.insert(entry[0], entry[1])
                    next_index = (next_index + 1) % self.size


                # Check load factor and resize if necessary
                if self._load_factor() <= 1 / 5 and self.size > 8:  # Minimum size of 8
                    self._resize(self.size // 2)
                return

            index = (index + 1) % self.size

    def get(self, key):
        index = myhash(key, self.size)

        while self.table[index] is not None:
            if self.table[index][0] == key:  # Found key
                return self.table[index][1]
            index = (index + 1) % self.size

        return None  # Key not found

    def __str__(self):
        return pprint.pformat(self.table)

# Define the hash function (from the previous code)
def myhash(key, size):  # key is a text string
    hash_value = 7  # better use a prime number
    for i in range(len(key)):
        hash_value = (hash_value * 31 + ord(key[i])) % size
    return hash_value

# Helper function to generate a random 3-digit key
def generate_random_key():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))

# Function to simulate one experiment
def run_experiment():
    hashtable = DynamicHashTable()
    expansions = 0
    contractions = 0

    # Generate a random binary string of length 100 (1 = insert, 0 = delete)
    binary_string = [random.choice([0, 1]) for _ in range(100)]

    # Track the table's size before each operation
    prev_size = len(hashtable.table)

    for action in binary_string:
        if action == 1: 
            key = generate_random_key()
            hashtable.insert(key, key)  # Insert key and value as the key
        elif action == 0 and hashtable.num_elements > 0:  # Delete 
            key = random.choice([entry[0] for entry in hashtable.table if entry is not None])  # Random key from the table
            hashtable.delete(key)

        # Track the current table size
        current_size = len(hashtable.table)

        # Count expansion and contraction
        if current_size > prev_size:
            expansions += 1
        elif current_size < prev_size:
            contractions += 1

        prev_size = current_size  # Update the previous size for next iteration

    return expansions, contractions


In [52]:
def main():
    total_expansions = 0
    total_contractions = 0
    expirements = 20

    # Run the experiment 20 times
    for _ in range(expirements):
        expansions, contractions = run_experiment()
        total_expansions += expansions
        total_contractions += contractions

    # Compute the average expansions and contractions
    average_expansions = total_expansions / 20
    average_contractions = total_contractions / 20

    # Output the results
    print(f"Average number of expansions: {average_expansions}")
    print(f"Average number of contractions: {average_contractions}")

# Run the main function
if __name__ == "__main__":
    main()

Average number of expansions: 1.7
Average number of contractions: 0.85
